## Twitter NLP


In [1]:
import json, os, string
import platform
from collections import Counter
from twitter_client import get_twitter_client
from tweepy import Cursor
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from selectfile import FileBrowser

In [2]:
client = get_twitter_client()
#print(client.me())

In [3]:
#twitter_user = input('Please introduce the twitter user name')

In [4]:
# print(twitter_user)
# cwd = os.getcwd()
# if os.path.isdir(os.path.join(cwd,'raw_data')):
#     fname = os.path.join(cwd, 'raw_data', 'user_timeline_{}.jsonl'.format(twitter_user)) 
# else:
#     fname = os.path.join(os.mkdir(os.path.join(cwd,'raw_data')), 'user_timeline_{}.jsonl'.format(twitter_user ))   
    
# print(fname)

In [5]:
# with open(fname, 'w') as f:
#     for page in Cursor(client.user_timeline, screen_name=twitter_user, count=200).pages(16):
#         for status in page:
#             f.write(json.dumps(status._json) + "\n")
   

In [6]:
raw_data_file = FileBrowser()
raw_data_file.widget()

## Tokenization

In [8]:
raw_data_file
raw_data_file.path

'C:\\Users\\Rodolfo\\Desktop\\NLP\\raw_data\\user_timeline_jeremycorbyn.jsonl'

In [9]:
def process(text, tokenizer=TweetTokenizer(), stopwords=[]):
    """Process the text of a tweet:
    - Lowercase
    - Tokenize
    - Stopword removal
    - Digits removal
    Return: list of strings
    """
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    return [tok for tok in tokens if tok not in stopwords and not tok.isdigit()]

In [10]:
tweet_tokenizer = TweetTokenizer()
punct = list(string.punctuation)
stopword_list = stopwords.words('english') + punct + ['rt', 'via', '...']
tf = Counter()
with open(raw_data_file.path, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tokens = process(text=tweet['text'], tokenizer=tweet_tokenizer, stopwords=stopword_list)
        tf.update(tokens)
for tag, count in tf.most_common(20):
    print("{}: {}".format(tag, count))

…: 2295
labour: 728
’: 433
today: 429
people: 330
government: 313
@uklabour: 300
tories: 247
nhs: 210
@theresa_may: 199
country: 166
us: 166
great: 153
tory: 144
vote: 143
#forthemany: 131
must: 130
years: 129
campaign: 127
many: 115
